In [1]:
import re, string, unicodedata, itertools, random, os, glob, math
import pickle

import numpy as np
import pandas as pd

In [2]:
def geraIndividuo(tamCromossomo):
    
    cromossomo = []
    
    while tamCromossomo > 0:
        cromossomo.append(random.randint(0,1))
        tamCromossomo -= 1
        
    return cromossomo

In [3]:
def geraPopulacao(tamPop, tamCromossomo):
    
    populacao = []
    
    while tamPop > 0:
        populacao.append(geraIndividuo(tamCromossomo))
        tamPop -= 1
        
    return populacao

In [4]:
#argumentos: recebe array binario do cromossomo (geraPopulacao[i])
#retorna: o valor real x e y do cromossomo

def cromossomoReal(cromossomo):
    
    x = 0
    y = 0
    i = 0
    
    while(i < len(cromossomo)/2):
        x += cromossomo[i] * pow(2, len(cromossomo)/2-i-1)
        i += 1
        
    while(i < len(cromossomo)):
        y += cromossomo[i] * pow(2, len(cromossomo)-i-1)
        i += 1

    return [x,y]

In [5]:
#argumentos: recebe um valor real e o tamanho do cromossomo (len(geraPopulacao[i]))
#retorna: o valor real x e y do cromossomo dentro do intervalo da funcao

def valorNoIntervalo(a, cromossomoLength):
    
    a = -5.12 + (10.24 / (pow(2,cromossomoLength / 2) - 1) * a)
    
    return a

In [15]:
#argumentos: recebe um cromossomo
#retorna: o valor real fitness do individuo

def fitness(cromossomo):
    
    valorReal = cromossomoReal(cromossomo)
    x = valorNoIntervalo(valorReal[0], len(cromossomo))
    y = valorNoIntervalo(valorReal[1], len(cromossomo))
    cosx = math.cos(2 * math.pi * x)
    cosy = math.cos(2 * math.pi * y)
    
    if(20 + pow(x,2) + pow(y,2) - 10*(cosx+cosy) == 0):
        resultado = 0
    else:
        resultado = 1/(20 + pow(x,2) + pow(y,2) - 10*(cosx+cosy))
    
    return resultado

In [7]:
#argumentos: a populacao toda
#retorna: o cromossomo de um individuo

def selecao(populacao):
    
    fitnessAcumulado = []
    fitnessTotal = 0
    i = 0
    
    while(i < len(populacao)):
        
        fitnessAtual = fitness(populacao[i])
        fitnessTotal += fitnessAtual
        fitnessAcumulado.append(fitnessTotal)
        i += 1
    
    selecionar = random.random()*fitnessTotal
    
    i = 0
    while(i < len(populacao)):
        if(fitnessAcumulado[i] >= selecionar):
            return populacao[i]
        else:
            i += 1

In [8]:
#argumentos: dois cromossomos (selecao(pop))
#retorno: um array contendo dois cromossomos

def crossover(pai, mae):
    
    ponto = random.randint(1,len(pai)-1)
    i = 0
    filhos = []
    filhoUm = []
    filhoDois = []
    
    while(i < len(pai)):
        if(i < ponto):
            filhoUm.append(pai[i])
            filhoDois.append(mae[i])
        else:
            filhoUm.append(mae[i])
            filhoDois.append(pai[i])
        i += 1
        
    filhos.append(filhoUm)
    filhos.append(filhoDois)
    
    return filhos

In [9]:
def novaGeracao(pop):
    
    novaPop = []
    
    while(len(novaPop) < len(pop)):
        filhos = crossover(selecao(pop), selecao(pop))
        novaPop.append(filhos[0])
        novaPop.append(filhos[1])
        
    return novaPop

In [10]:
def fitnessGeracao(pop):
    
    fitnessGeracao = 0
    i = 0
    
    while(i < len(pop)):
        fitnessGeracao += fitness(pop[i])
        i += 1
    
    return fitnessGeracao

In [13]:
pop = geraPopulacao(100, 32)
i = 0
j = 0
maior = 0
maiorGen = 0
while(j < 40):
    while(i < len(pop)):
        fitAtual = fitness(pop[i])
        if(fitAtual > maior):
            print("Novo maior fitness:", fitAtual, 'Antigo:', maior)
            maior = fitAtual
            maiorGen = j
        i += 1
    print('Gen', j, 'Fitness total:', fitnessGeracao(pop))
    pop = novaGeracao(pop)
    j += 1
    
print('Geracao', maiorGen, 'Cromossomo', pop[i-1], 'Fitness', maior)

Novo maior fitness: 5.5700925203087275 Antigo: 0
Novo maior fitness: 19.837600819457823 Antigo: 5.5700925203087275
Novo maior fitness: 28.482344497651184 Antigo: 19.837600819457823
Gen 0 Fitness total: 354.30663750753536
Gen 1 Fitness total: 578.5051534131272
Gen 2 Fitness total: 1123.7366461095185
Gen 3 Fitness total: 1808.8347328571404
Gen 4 Fitness total: 2283.5569857415426
Gen 5 Fitness total: 2451.153706661238
Gen 6 Fitness total: 3139.4919886730936
Gen 7 Fitness total: 7960.072713609675
Gen 8 Fitness total: 208751.19473946095
Gen 9 Fitness total: 497144.5417767616
Gen 10 Fitness total: 759768.0699233082
Gen 11 Fitness total: 1212238.7189507233
Gen 12 Fitness total: 2059381.3233339912
Gen 13 Fitness total: 2420015.3657839913
Gen 14 Fitness total: 2747785.7019449784
Gen 15 Fitness total: 3181022.7655324275
Gen 16 Fitness total: 3877499.01159381
Gen 17 Fitness total: 4813378.769856723
Gen 18 Fitness total: 5936472.856000402
Gen 19 Fitness total: 7940000.6524655
Gen 20 Fitness total:

In [16]:
cr = (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1)
valorReal = cromossomoReal(cr)
x = valorNoIntervalo(valorReal[0], len(cr))
y = valorNoIntervalo(valorReal[1], len(cr))
print(valorReal, x, y)
print(fitness(cr))

[32771.0, 32759] 0.0005468833447777044 -0.0013281452658882031
2443.2496359735032
